### Linear Regression & Neural Networks

Dataset: MNIST


In [1]:
# change the temp directory from user/app data ... to E: drive to avoid IO related issues.
from urllib import request
request.tempfile.tempdir = 'E:/tensorflow/temp'

In [2]:
from tensorflow.examples.tutorials.mnist import input_data, mnist_softmax
mnist = input_data.read_data_sets("E:\\tensorflow", one_hot=True)
#data_sets = input_data.read_data_sets(FLAGS.train_dir, FLAGS.fake_data)

Extracting E:\tensorflow\train-images-idx3-ubyte.gz
Extracting E:\tensorflow\train-labels-idx1-ubyte.gz
Extracting E:\tensorflow\t10k-images-idx3-ubyte.gz
Extracting E:\tensorflow\t10k-labels-idx1-ubyte.gz


#### Implementing Regression

In [3]:
import tensorflow as tf

# define placeholder for input
x = tf.placeholder(tf.float32, [None, 784])

# define weights and bias
W = tf.Variable(tf.zeros([784,10]),dtype=tf.float32)
b = tf.Variable(tf.zeros([10]))

# define regression model y = Wx + b; apply softmax for classification
y = tf.nn.softmax(tf.matmul(x, W) + b)

# define placeholder for labels
y_ = tf.placeholder(tf.float32, [None, 10])

# define cross-entry loss function
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# define optimizer to minimize loss
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# initialize variables
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

#### Model Training
    
    - Using loss function called "cross-entropy"

In [5]:
# training model for 1000 steps with batch of 100
for _ in range(1000):
    # get next batch
    batch_xs, batch_ys = mnist.train.next_batch(100)
    # train model using current input batch
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})    

#### Evaluation

In [6]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


### Feedforward Neural Network Classifier

In [18]:
import tensorflow as tf

# define variables
num_input = 784
num_classes = 10

learning_rate = 0.01

n_hidden_1 = 400
n_hidden_2 = 100

# define placeholder for input and labels
x = tf.placeholder(tf.float32, [None, num_input])
y_ = tf.placeholder(tf.float32, [None, num_classes])

# layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# construct model [784 -> 500 -> 200 -> 10]
layer0 = x
layer1 = tf.nn.sigmoid(tf.matmul(layer0, weights['h1']) + biases['b1'])   # here using sigmoid increases accuracy greatly from no activation or using relu
layer2 = tf.nn.sigmoid(tf.matmul(layer1, weights['h2']) + biases['b2'])
output_layer = tf.matmul(layer2, weights['out']) + biases['out']

# define cross entropy loss function
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_layer, labels=y_))

# define adam as optimizer
 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

# minimize on each training step
train = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [19]:
# initialize variables
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# train model
for step in range(2001):
    # get training batch
    batch_x, batch_y = mnist.train.next_batch(100)
    # train model on training batch
    sess.run(train,feed_dict={x:batch_x, y_:batch_y})
    
    if step % 100 == 0 or step == 1:
        loss, acc = sess.run([loss_op, accuracy], feed_dict={x: batch_x, y_: batch_y})
        print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(loss) + ", Training Accuracy= " + "{:.3f}".format(acc))

print("Optimization Finished!")

# Calculate accuracy for MNIST test images
print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Step 0, Minibatch Loss= 5.4860, Training Accuracy= 0.110
Step 1, Minibatch Loss= 4.2184, Training Accuracy= 0.190
Step 100, Minibatch Loss= 0.4732, Training Accuracy= 0.870
Step 200, Minibatch Loss= 0.4730, Training Accuracy= 0.850
Step 300, Minibatch Loss= 0.3066, Training Accuracy= 0.900
Step 400, Minibatch Loss= 0.2163, Training Accuracy= 0.940
Step 500, Minibatch Loss= 0.1975, Training Accuracy= 0.950
Step 600, Minibatch Loss= 0.1794, Training Accuracy= 0.950
Step 700, Minibatch Loss= 0.0954, Training Accuracy= 0.970
Step 800, Minibatch Loss= 0.2783, Training Accuracy= 0.950
Step 900, Minibatch Loss= 0.1041, Training Accuracy= 0.970
Step 1000, Minibatch Loss= 0.3376, Training Accuracy= 0.910
Step 1100, Minibatch Loss= 0.1438, Training Accuracy= 0.950
Step 1200, Minibatch Loss= 0.0451, Training Accuracy= 1.000
Step 1300, Minibatch Loss= 0.2090, Training Accuracy= 0.960
Step 1400, Minibatch Loss= 0.0984, Training Accuracy= 0.960
Step 1500, Minibatch Loss= 0.1719, Training Accuracy= 0